In [30]:
import numpy as np
from collections import Counter
from math import log2

In [31]:
class Node(object): #DecisionTreeNode CLass
    def __init__(self, feature,feature_val,predict_val):
        self.feature = feature
        self.feature_val = feature_val
        self.predict_val = predict_val
        self.children = []

    def add_child(self, obj):
        self.children.append(obj)

In [44]:
class MyDecisionTreeClassifier:
    def entropy(self,p): #Entropy Function
        entropy = 0
        for pi in p:
            entropy+=(-1)*pi*log2(pi)
        return entropy

    def info(self,p): #Info for a node or after split ; p-2d array
        info = 0
        total_before_split = p.sum()
        for i in range(len(p)):
            total = p[i].sum()
            prob =[]
            for j in range(len(p[i])):
                if(p[i][j]==0):
                    continue
                prob.append(p[i][j]/total)
            entropy_val = self.entropy(prob)
            info += (entropy_val*total)/total_before_split
        return info
    
    def buildTree(self,x_train,y_train,features,level,f_used,print_True): #Used to build tree and print Tree
        
        values = Counter(y_train)
        entropy = self.info(np.array([[x for x in Counter(y_train).values()]]))
        predict_val = max(values, key=values.get)   #Majority Value at a Node
        
        if(print_True): #Prints Tree Structure if Print_True=True
            print("Level ",level)
            for i in values.keys():
                    print("Count of ",data_set.target_names[i]," = ",values[i])
            print("Current Entropy  is = ", entropy)
            if(entropy==0 or len(features)==0):
                print("Reached leaf Node")
                print(" ")
                
        if(entropy==0 or len(features)==0): # Leaf Node Conditions
            return Node(-1,-1,predict_val)
        
        f_max,f_max_val,gain_ratio_max =  self.find_feature_for_split(x_train,y_train,features,f_used) #finds the best feature and feature value to split Current Node
        n = Node(f_max,f_max_val,predict_val)
        f_used.append(f_max)
        
        if(print_True):
            print("Splitting on feature ",data_set.feature_names[f_max], "<=" ,f_max_val," with gain ratio ",gain_ratio_max)
            print(" ")
        
        less_equal = np.where(x_train[:,f_max] <= f_max_val)
        greater = np.where(x_train[:,f_max] > f_max_val)
        #left child Data
        le_y_train = [y_train[x] for x in less_equal]
        le_x_train = [x_train[x] for x in less_equal]
        #right child Data
        g_y_train = [y_train[x] for x in greater]
        g_x_train = [x_train[x] for x in greater]
        
        n.add_child(self.buildTree(le_x_train[0],le_y_train[0],features,level+1,f_used,print_True)) #Adding Left node to children list
        n.add_child(self.buildTree(g_x_train[0],g_y_train[0],features,level+1,f_used,print_True)) #Adding right node to children list
        return n
    
    def find_feature_for_split(self,x_train,y_train,features,f_used): #Function to find best feature to split current Node
        info_Node = self.info(np.array([[x for x in Counter(y_train).values()]]))
        gain_ratio_max = -100000
        f_max = -1
        f_max_val = -1
        for f in range(features.shape[0]):
            gain_ratio_f_max,split_f_val = self.continuous_feature_split(x_train,y_train,features[f],info_Node)
            if((gain_ratio_max <= gain_ratio_f_max) or f==0):
                if(gain_ratio_max == gain_ratio_f_max and f in f_used):
                    continue
                gain_ratio_max = gain_ratio_f_max
                f_max = features[f]
                f_max_val = split_f_val
                
        return f_max,f_max_val,gain_ratio_max
    
    def continuous_feature_split(self,x_train,y_train,f,info_Node): # Function to find best continuous valued feature value to split node
        feature_values = np.copy(x_train[:,f])
        feature_values.sort()
        total= len(y_train)
        gain_ratio_f_max = np.float64(-100000) #initializing gain_ratio_max
        split_f_val = x_train[0,f]
        # There are two possible split less than Equal to and greater than
        # We iterate over all middle points between feature f sorted values and tries to find the value that Maximize gain Ratio
        # GainRatio = Info_Gain/Split_Info
        # Info_gain = Info_Node(entropy of Node) - Info_F(weighted mean of entropy post split)
        # Split_Info is -D1/D*Log(D1/D2) -D2/DLog(D2/D) where D1 and D2 are data points count post split in 2 child Nodes(D=D1+D2)  
        for x in range(len(feature_values)-1): #iterating over all sorted values of feature f
            info_f = np.float64(0)
            less_than_eqaul_split =  Counter([y_train[x] for x in np.where(x_train[:,f] <= ((feature_values[x]+feature_values[x+1])/2))[0]])
            less_than_eqaul_split = np.array([[x for x in less_than_eqaul_split.values()]])
            info_f = info_f +  (less_than_eqaul_split.sum())*(self.info(less_than_eqaul_split))/(total)
            D1 = less_than_eqaul_split.sum()
            
            if(info_f!=info_f):
                info_f = np.float64(0)

            greater_than_split = Counter([y_train[x] for x in np.where(x_train[:,f] > ((feature_values[x]+feature_values[x+1])/2))[0]])
            greater_than_split = np.array([[x for x in greater_than_split.values()]])
            info_f = info_f +((greater_than_split.sum())*(self.info(greater_than_split))/(total))
            D2 = greater_than_split.sum()
            gain_ratio_f =  (info_Node -info_f)/(self.info(np.array([[D1,D2]]))) 
            if(gain_ratio_f >= gain_ratio_f_max):
                gain_ratio_f_max = gain_ratio_f
                split_f_val = (feature_values[x]+feature_values[x+1])/2
        return (gain_ratio_f_max,split_f_val)
    
    def predict_x(self,x,n): #Predict Class for a particular data in x_test
        if(n.feature == -1 ):
            return n.predict_val
        else:
            if(x[n.feature]<=n.feature_val):
                return self.predict_x(x,n.children[0])
            else:
                return self.predict_x(x,n.children[1])
    def predict(self,x_test): #Predict CLass for all data in X_test
        y_predict = []
        for xi in x_test:
            y_predict.append(self.predict_x(xi,self.n))
        return y_predict
    
    def fit(self,x_train,y_train,print_True=False): #fit function for training algo DTC also to Print Tree
        f = [i for i in range(x_train.shape[1])]
        x_train = np.array(x_train)
        y_train = np.array(y_train)
        f = np.array(f)
        self.n = self.buildTree(x_train,y_train,f,0,[],print_True)
    
    def score(self,y_predict,y_test): #Used to calculate score
        total = len(y_predict)
        correct = 0
        for i in range(len(y_predict)):
            if(y_predict[i]==y_test[i]):
                correct +=1
        return correct/total


In [38]:
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [39]:
data_set = datasets.load_iris()
x_train,x_test,y_train,y_test = train_test_split(data_set.data,data_set.target,random_state = 0)

In [45]:
clf = MyDecisionTreeClassifier()
clf.fit(x_train,y_train,True)
y_predict = clf.predict(x_test)
clf.score(y_predict,y_test)

Level  0
Count of  versicolor  =  34
Count of  virginica  =  41
Count of  setosa  =  37
Current Entropy  is =  1.5807197138422102
Splitting on feature  petal width (cm) <= 0.8  with gain ratio  0.9999999999999998
 
Level  1
Count of  setosa  =  37
Current Entropy  is =  0.0
Reached leaf Node
 
Level  1
Count of  versicolor  =  34
Count of  virginica  =  41
Current Entropy  is =  0.9937071066045079
Splitting on feature  petal length (cm) <= 4.95  with gain ratio  0.7064313543656205
 
Level  2
Count of  versicolor  =  33
Count of  virginica  =  3
Current Entropy  is =  0.41381685030363374
Splitting on feature  petal width (cm) <= 1.65  with gain ratio  0.643158422044091
 
Level  3
Count of  versicolor  =  32
Current Entropy  is =  0.0
Reached leaf Node
 
Level  3
Count of  virginica  =  3
Count of  versicolor  =  1
Current Entropy  is =  0.8112781244591328
Splitting on feature  sepal width (cm) <= 3.1  with gain ratio  1.0
 
Level  4
Count of  virginica  =  3
Current Entropy  is =  0.0
R

C:\Users\Sanchit Gupta\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


0.9736842105263158